# 9章 データの集約とグループ演算
## 9.1 GroupByの仕組み

In [79]:
from __future__ import division
from numpy.random import randn
import numpy as np
from io import StringIO 
import os
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [80]:
# グループ関数を使用するためDataFrameを作成
df = DataFrame({'key1':['a','a','b','b','a'], 
              'key2':['one', 'two', 'one', 'two', 'one'], 
              'data1':np.random.randn(5), 
              'data2':np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.204708,1.393406,a,one
1,0.478943,0.092908,a,two
2,-0.519439,0.281746,b,one
3,-0.555730,0.769023,b,two
4,1.965781,1.246435,a,one


In [81]:
# groupby関数でグループ化
# groupby関数はDataFrameクラスの関数
# 引数：グループ化のKeyとなる列名
grouped = df['data1'].groupby(df['key1'])
grouped

In [82]:
# グループ化したKeyごとの平均値
# 引数なしは一番左のデータを出力
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [83]:
# groupby関数に複数のKeyを指定
# 複数の引数を指定すると左の引数からグループ化
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [84]:
# unstack関数で縦と横の切替
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [85]:
# 配列をKeyとしてグループ化する前にdf変数の確認
df

,data1,data2,key1,key2
0,-0.204708,1.393406,a,one
1,0.478943,0.092908,a,two
2,-0.519439,0.281746,b,one
3,-0.555730,0.769023,b,two
4,1.965781,1.246435,a,one


In [86]:
# 長さが同じ配列をKeyにグループ化することができる
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [87]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


In [88]:
# グループ化したデータの件数を確認
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 9.1.1 グループをまたいだ繰り返し

In [89]:
# グループ化したKeyごとのデータ
# name：グループ化したKey
# group：グループ化したデータ
for name, group in df.groupby(['key1', 'key2']):
    print(name)
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -0.204708  1.393406    a  one
4  1.965781  1.246435    a  one
('a', 'two')
      data1     data2 key1 key2
1  0.478943  0.092908    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.519439  0.281746    b  one
('b', 'two')
     data1     data2 key1 key2
3 -0.55573  0.769023    b  two


In [90]:
df

,data1,data2,key1,key2
0,-0.204708,1.393406,a,one
1,0.478943,0.092908,a,two
2,-0.519439,0.281746,b,one
3,-0.555730,0.769023,b,two
4,1.965781,1.246435,a,one


In [91]:
# Keyごとの全ての行をリスト化
list(df.groupby('key1'))

[('a',       data1     data2 key1 key2
  0 -0.204708  1.393406    a  one
  1  0.478943  0.092908    a  two
  4  1.965781  1.246435    a  one), ('b',       data1     data2 key1 key2
  2 -0.519439  0.281746    b  one
  3 -0.555730  0.769023    b  two)]

In [92]:
# リストをKeyごとの辞書に変換
pieces = dict(list(df.groupby('key1')))
pieces

{'a':       data1     data2 key1 key2
 0 -0.204708  1.393406    a  one
 1  0.478943  0.092908    a  two
 4  1.965781  1.246435    a  one, 'b':       data1     data2 key1 key2
 2 -0.519439  0.281746    b  one
 3 -0.555730  0.769023    b  two}

In [93]:
# Key1がBのデータ取得
pieces['b']

,data1,data2,key1,key2
2,-0.519439,0.281746,b,one
3,-0.555730,0.769023,b,two


In [94]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

### 9.1.3 ディクショナリ形式やシリーズのグルーピング

In [95]:
# 指定したcolumとindexを持つ5行5列のDataFrame作成
people = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
# 2行目から3行未満のb,c列にNull値を代入
people.ix[2:3, ['b', 'c']] = np.nan
people

C:\WinPython\python-3.6.5.amd64\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [96]:
# DataFrameと突合わせするためのディクショナリを作成
mapping = {'a':'red', 'b':'red', 'c':'blue', 
          'd':'blue', 'e':'red', 'f':'orange'}

In [97]:
# 上記ディクショナリのKeyとDataFrameの列名が一致した際のValueの値でグループ化
# redはa・b・e列でグループ化
# blueはc・d列でグループ化
# orangeはf列をグループ化　⇒　f列は無いのでグループ化しないが、エラーにはならない
by_column = people.groupby(mapping, axis=1)

In [98]:
# 各色の合計値を集計
by_column.sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


### 9.1.4 関数を用いたグルーピング

In [99]:
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [100]:
# indexの文字の長さでグルーピング ⇒ 上記名前の長さ
people.groupby(len).sum()

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [101]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

## 9.2 データの集約

In [102]:
df

,data1,data2,key1,key2
0,-0.204708,1.393406,a,one
1,0.478943,0.092908,a,two
2,-0.519439,0.281746,b,one
3,-0.555730,0.769023,b,two
4,1.965781,1.246435,a,one


In [103]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

In [104]:
## 自分で作った集約関数を使うには、aggregateメソッド(aggメソッド)に渡す
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [105]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

### 9.3.1 applyメソッド

In [106]:
def top(df, n=5, column='tip_pct'):
    return df.sort_index(by=column)[-n:]
top(tips, n=6)

NameError: name 'tips' is not defined

In [107]:
tips.groupby('smoker').apply(top)

NameError: name 'tips' is not defined

In [72]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

### 9.3.3 例：グループ毎に指定する値で欠損値を埋める

In [62]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.265934
2         NaN
3   -1.063512
4         NaN
5   -2.359419
dtype: float64

In [63]:
s.fillna(s.mean())

0   -1.562955
1   -1.265934
2   -1.562955
3   -1.063512
4   -1.562955
5   -2.359419
dtype: float64

In [64]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.199543
New York     -1.541996
Vermont            NaN
Florida      -1.307030
Oregon        0.286350
Nevada             NaN
California   -0.753887
Idaho              NaN
dtype: float64

In [65]:
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [66]:
data.groupby(group_key).mean()

East   -1.016190
West   -0.233768
dtype: float64

In [67]:
fill_mean = lambda g: g.fillna(g.mean())
fill_mean

<function __main__.<lambda>>

In [68]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.199543
New York     -1.541996
Vermont      -1.016190
Florida      -1.307030
Oregon        0.286350
Nevada       -0.233768
California   -0.753887
Idaho        -0.233768
dtype: float64

In [69]:
fill_values = {'East':0.5, 'West':-1}

In [70]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [71]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.199543
New York     -1.541996
Vermont       0.500000
Florida      -1.307030
Oregon        0.286350
Nevada       -1.000000
California   -0.753887
Idaho        -1.000000
dtype: float64

## 9.4 クロス集計

### 9.4.1 crosstabメソッド：クロス集計

In [74]:
# クロス集計とは、カテゴリごとのサンプル数（出現回数、出現頻度）
# クロス集計するために文字列を作成
from StringIO import StringIO
data = """\
Sample    Gender    Handedness
1    Female    Right-handed
2    Male    Left-handed
3    Female    Right-handed
4    Male    Right-handed
5    Male    Left-handed
6    Male    Right-handed
7    Female    Right-handed
8    Female    Left-handed
9    Male    Right-handed
10    Female    Right-handed"""
# スペース区切りでテーブルの作成
# read_table関数の戻り値はDatraFrame型
data = pd.read_table(StringIO(data), sep='\s+')

ModuleNotFoundError: No module named 'StringIO'

In [73]:
data

Ohio         -0.199543
New York     -1.541996
Vermont            NaN
Florida      -1.307030
Oregon        0.286350
Nevada             NaN
California   -0.753887
Idaho              NaN
dtype: float64